<a href="https://colab.research.google.com/github/Angelica2929/sales_prediction/blob/main/Preprocesamiento_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Proyecto 1 Prediccion Ventas - Parte 5 (base)**

El objetivo de este paso es ayudar al distribuidor usando el aprendizaje automático para realizar predicciones sobre futuras ventas basándose en los datos proporcionados.



1.Importar las biblioetcas necesarias.

In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

2.Cargar estructura de datos

In [40]:
filename = '/content/drive/MyDrive/CODING DOJO/WEEK 5/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


3. Explorar los datos

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Al realizar la exploracion de nuestra data, podemos identificar 3 tipos de caracteristicas asi:


**NUMERICAS**

'Item_Weight',
'Item_Fat_Content',
'Item_Visibility',
'Item_MRP',
'Outlet_Establishment_Year'

**ORDINALES**.                                              
'Item_Fat_Content'

**NOMINALES**

'Item_Identifier',
'Item_Type',
'Outlet_Identifier',
'Outlet_Size',
'Outlet_Location_Type',
'Outlet_Type'

Para el proceso de aprendizaje de nuestro modelo sera necesario transformar nuestras caracteristicas nominales y si es el caso ordinales en variables numericas, esto para mayor funcionalidad de nuestro modelo.

Adicionalmente identificamos datos faltantes en las categorias de Item_Weight (Variable Numerica)y OutleT Size (Variable Categorica), situacion que debemos abordar a continuacion con el metodo de Mean Imputation para variables numericas y Most_frequent imputation para variables categoricas.

4. Codificación ordinal

In [42]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [43]:
# Codificación ordinal "Item_Fat_Content"
replacement_dictionary = {'Low':1, 'Low Fat':1, 'LF':1, 'low fat':1, 'Regular':0, 'reg':0}
df['Item_Fat_Content'].replace(replacement_dictionary, inplace=True)
df['Item_Fat_Content']

0       1
1       0
2       1
3       0
4       1
       ..
8518    1
8519    0
8520    1
8521    0
8522    1
Name: Item_Fat_Content, Length: 8523, dtype: int64

In [44]:
df['Item_Fat_Content'].value_counts()

1    5517
0    3006
Name: Item_Fat_Content, dtype: int64

Nuestra variable Item Fat Content es una variable de tipo nominal, fue necesario una conversion, por lo cual hicimos  un diccionario y un replace para ajustar la informacion con inconformidades. Ahora esta ordinalmente codificada.

5. Train test Split

In [45]:
X = df.drop('Item_Outlet_Sales', axis=1)
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Nuestra variable target (a predecir) es **Item_Outlet_Sales** que corresponde al total de ventas por tienda, para predecir dicha variable nos apoyaremos basados en el resto de categorias que vienen incluidas como peso, contenido graso,categoria, precio, año creacion tienda etc...

Este proceso consiste en dividir nuestro conjunto de datos en 2 grupos (entrenamiento y prueba), y es aqui donde ocurre el proceso de aprendizaje y validacion de nuestro modelo, esto con el proposito de conocer como se va a comportar nuestro target en la medida que nuestro conjunto de caracteristicas se mueva de una o determinada forma.


6.Instanciar selectores de columnas

In [46]:
cat_selector = make_column_selector(dtype_include='object')
cat_selector

In [47]:
#Nos devuelve una lista con las columnas que coinciden con el patron solicitado, en este caso, las categoricas.
cat_selector(X_train)

['Item_Identifier',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [55]:
num_selector = make_column_selector(dtype_include='number')
num_selector

In [56]:
#Nos devuelve una lista con las columnas que coinciden con el patron solicitado, en este caso, las numericas.
num_selector(X_train)

['Item_Weight',
 'Item_Fat_Content',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year']

7.Instanciar transformadores

In [51]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

Tenemos 2 simple imputers con diferentes estrategias de imputacion, most frequent y mean imputer.

8. Instanciar pipelines

In [53]:
#Pipeline Numerico
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [54]:
# Pipeline Categorico
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

9. Instanciar ColumnTransformer

In [62]:
# Tuplas para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcc369304d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcc369309d0>)])

Las tuplas lo que hacen es que nuestros transformadores coincidan con los diferentes tipos de datos de nuestras columnas, y haremos uso de esos transformdores a traves de pipelines.

10.Transformador de datos

In [58]:
# fit on train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcc369304d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcc369309d0>)])

El codigo anterior nos permite ajustar el column transformer en nuestro espacio de entrenamiento, ajustando nuestros transformadores dentro del COLUMN TRANSFORMER.

In [60]:
# Transformar datos de entrenamiento y de prueba
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

11.Inspeccionar el resultado

In [61]:
print(np.isnan(X_train_processed).sum().sum(), 'valores faltantes en datos de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'valores faltantes en datos de prueba')
print('\n')
print('Toda la data en X_train_processed es', X_train_processed.dtype)
print('Toda la data en X_test_processed es', X_test_processed.dtype)
print('\n')
print('shape de la data es', X_train_processed.shape)
print('\n')
X_train_processed

0 valores faltantes en datos de entrenamiento
0 valores faltantes en datos de prueba


Toda la data en X_train_processed es float64
Toda la data en X_test_processed es float64


shape de la data es (6392, 1591)




array([[ 0.81724868,  0.7403206 , -0.71277507, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5563395 , -1.35076614, -1.29105225, ...,  0.        ,
         1.        ,  0.        ],
       [-0.13151196, -1.35076614,  1.81331864, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.11373638,  0.7403206 , -0.92052713, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.76600931,  0.7403206 , -0.2277552 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81724868,  0.7403206 , -0.95867683, ...,  1.        ,
         0.        ,  0.        ]])

Como podemos leerlo en el array de numpy arrojado, no hay valores faltantes, todos los datos son de tipo numerico flotante y despues del one hot coder tenemos mas columnas que al inicio, ahora tenemos 1591 vs 12 columnas iniciales, todas fueron escaladas.


Realizamos varias tranformaciones en un solo paso de transformacion, acortando el codigo.